In [ ]:
import cv2
import datetime
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.15.0


In [ ]:
BATCH_SIZE = 32
EPOCHS = 25
IM_SIZE_W = 64
IM_SIZE_H = 64

AUTOTUNE = tf.data.experimental.AUTOTUNE

tf.random.set_seed(10)

In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset'):
    print(dirname)

/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset
/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/NORMAL
/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/PNEUMONIA
/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/COVID


In [ ]:
filenames = tf.io.gfile.glob('/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/*/*')
print(len(filenames))
filenames[:3]

5228


['/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/PNEUMONIA/PNEUMONIA_1740.png',
 '/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/PNEUMONIA/PNEUMONIA_1675.png',
 '/content/drive/MyDrive/Kuliah/Machine Learning/Tubes Dataset/PNEUMONIA/PNEUMONIA_1622.png']

In [ ]:
data = pd.DataFrame()
for el in range(0, len(filenames)):
    target = filenames[el].split('/')[-2]
    path = filenames[el]

    data.loc[el, 'filename'] = path
    data.loc[el, 'class'] = target

print(data['class'].value_counts(dropna=False))
data

NORMAL       1802
PNEUMONIA    1800
COVID        1626
Name: class, dtype: int64


,filename,class
0,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
1,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
2,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
3,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
4,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
...,...,...
5223,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
5224,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
5225,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
5226,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID


In [ ]:
data = shuffle(data, random_state=42)
data.reset_index(drop=True, inplace=True)
data

,filename,class
0,/content/drive/MyDrive/Kuliah/Machine Learning...,NORMAL
1,/content/drive/MyDrive/Kuliah/Machine Learning...,NORMAL
2,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
3,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
4,/content/drive/MyDrive/Kuliah/Machine Learning...,PNEUMONIA
...,...,...
5223,/content/drive/MyDrive/Kuliah/Machine Learning...,NORMAL
5224,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
5225,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID
5226,/content/drive/MyDrive/Kuliah/Machine Learning...,COVID


In [ ]:
change = {'PNEUMONIA' : '0',
'NORMAL' : '1',
'COVID' : '2',
}

data['class'] = data['class'].map(change)
data

,filename,class
0,/content/drive/MyDrive/Kuliah/Machine Learning...,1
1,/content/drive/MyDrive/Kuliah/Machine Learning...,1
2,/content/drive/MyDrive/Kuliah/Machine Learning...,2
3,/content/drive/MyDrive/Kuliah/Machine Learning...,0
4,/content/drive/MyDrive/Kuliah/Machine Learning...,0
...,...,...
5223,/content/drive/MyDrive/Kuliah/Machine Learning...,1
5224,/content/drive/MyDrive/Kuliah/Machine Learning...,2
5225,/content/drive/MyDrive/Kuliah/Machine Learning...,2
5226,/content/drive/MyDrive/Kuliah/Machine Learning...,2


In [ ]:
indexes=[]

def func(x):
    if x[-4:] != '.png':
        idx = data[data['filename'] == x].index
        indexes.append(idx[0])
        print(idx[0], x)
    return x

data['filename'].map(func)

print(data.shape)
data.drop(index=indexes, axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)
print(data.shape)

(5228, 2)
(5228, 2)


In [ ]:
for el in range(120, 100):
    path = data.loc[el, 'filename']
    img  = cv2.imread(path)
    print(img.shape)

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42, stratify=data['class'])
print(train_data['class'].value_counts(dropna=False))
print(val_data['class'].value_counts(dropna=False))

1    1622
0    1620
2    1463
Name: class, dtype: int64
0    180
1    180
2    163
Name: class, dtype: int64


In [ ]:
train_data, test_data = train_test_split(train_data, test_size=0.1, random_state=42, stratify=train_data['class'])
print(train_data['class'].value_counts(dropna=False))
print(test_data['class'].value_counts(dropna=False))

1    1460
0    1458
2    1316
Name: class, dtype: int64
1    162
0    162
2    147
Name: class, dtype: int64


In [ ]:
print(train_data)

                                               filename class
4921  /content/drive/MyDrive/Kuliah/Machine Learning...     1
4160  /content/drive/MyDrive/Kuliah/Machine Learning...     0
3378  /content/drive/MyDrive/Kuliah/Machine Learning...     0
4261  /content/drive/MyDrive/Kuliah/Machine Learning...     1
3779  /content/drive/MyDrive/Kuliah/Machine Learning...     2
...                                                 ...   ...
886   /content/drive/MyDrive/Kuliah/Machine Learning...     2
3736  /content/drive/MyDrive/Kuliah/Machine Learning...     1
2919  /content/drive/MyDrive/Kuliah/Machine Learning...     1
1695  /content/drive/MyDrive/Kuliah/Machine Learning...     2
362   /content/drive/MyDrive/Kuliah/Machine Learning...     0

[4234 rows x 2 columns]


In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,
                             zoom_range=0.1, # 0.05
                             brightness_range=[0.9, 1.0],
                             height_shift_range=0.05,
                             width_shift_range=0.05,
                             rotation_range=10,
                             )


test_datagen = ImageDataGenerator(rescale = 1./255)

train_gen = datagen.flow_from_dataframe(train_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode='grayscale',
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=True,
                                        num_parallel_calls=AUTOTUNE)

val_gen = test_datagen.flow_from_dataframe(val_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode='grayscale',
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=False,
                                        num_parallel_calls=AUTOTUNE)

test_gen = test_datagen.flow_from_dataframe(test_data,
                                        x_col="filename",
                                        y_col="class",
                                        target_size=(IM_SIZE_W, IM_SIZE_H),
                                        color_mode='grayscale',
                                        batch_size=BATCH_SIZE,
                                        class_mode='categorical',
                                        shuffle=False,
                                        num_parallel_calls=AUTOTUNE)

Found 4234 validated image filenames belonging to 3 classes.
Found 523 validated image filenames belonging to 3 classes.
Found 471 validated image filenames belonging to 3 classes.


In [ ]:
classes = np.unique(train_data['class'])
classes

array(['0', '1', '2'], dtype=object)

In [ ]:
from sklearn.utils import class_weight


class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                  classes = np.unique(train_data['class']),
                                                  y= train_data['class'])
class_weights = dict(enumerate(class_weights))

class_weights

{0: 0.9679926840420667, 1: 0.9666666666666667, 2: 1.0724417426545085}

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Add

def Residual_Unit(input, in_channel, out_channel, stride=1):

    # initialize as the input (identity) data
    shortcut = input
    shortcut = Conv2D(out_channel, (1, 1), padding='same', strides=stride)(shortcut)

    # RestNet module
    x = BatchNormalization()(input)
    x = Activation('relu')(x)
    x = Conv2D(in_channel, (1, 1))(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(in_channel, (3, 3), padding='same', strides=stride)(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(out_channel, (1, 1), padding='same')(x)

    # identity
    x = Add()([x, shortcut])

    return x

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Multiply

def Attention_Block(input, skip):

    # initial Attention Module parameters
    p = 1
    t = 2
    r = 1
    skip_connections = []
    # calculate input and output channel based on previous layers
    in_channel = input.shape[-1]
    out_channel = in_channel

    # pre-activation Residual Unit
    for _ in range(p):
        x = Residual_Unit(input, in_channel, out_channel)

    # Trunk Branch
    for _ in range(t):
        Trunck_output = Residual_Unit(x, in_channel, out_channel)

    # Soft Mask Branch
    ## 1st down sampling
    x = MaxPooling2D(padding='same')(x)
    for _ in range(r):
        x = Residual_Unit(x, in_channel, out_channel)

    if x.shape[1] % 4 == 0:
        for i in range(skip-1):
            ## skip connections
            skip_connections.append(Residual_Unit(x, in_channel, out_channel))

            ## 2rd down sampling
            x = MaxPooling2D(padding='same')(x)
            for _ in range(r):
                x = Residual_Unit(x, in_channel, out_channel)

        skip_connections = list(reversed(skip_connections))

        for i in range(skip-1):
            ## 1st up sampling
            for _ in range(r):
                x = Residual_Unit(x, in_channel, out_channel)
            x = UpSampling2D()(x)

            # skip connections
            x = Add()([x, skip_connections[i]])

    ## 2rd up samplping
    for i in range(r):
        x = Residual_Unit(x, in_channel, out_channel)
    x = UpSampling2D()(x)

    ## output
    x = Conv2D(out_channel, (1, 1))(x)
    x = Conv2D(out_channel, (1, 1))(x)
    soft_mask_output = Activation('sigmoid')(x)

    # Attention: (1 + soft_mask_output) * Trunck_output
    soft_mask_output = Lambda(lambda x: x + 1)(soft_mask_output)
    output = Multiply()([soft_mask_output, Trunck_output])

    # Last Residual Block
    for i in range(p):
        output = Residual_Unit(output, in_channel, out_channel)

    return output

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

def AttentionResNet92(shape, in_channel, kernel_size, n_classes, dropout=None, regularization=0.01):

    input_data = Input(shape=shape)  # 32x32x3
    x = Conv2D(in_channel, kernel_size=kernel_size, padding='same')(input_data)  # 32x32x32
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2, padding='same')(x)  # 16x16x32

    out_channel = in_channel * 4  # 256
    x = Residual_Unit(x, in_channel, out_channel)  # 16x16x128
    x = Attention_Block(x, skip=2)

    in_channel = out_channel  # 64
    out_channel = in_channel * 4  # 256
    x = Residual_Unit(x, in_channel, out_channel, stride=2)  # 8x8x256
    x = Attention_Block(x, skip=1)
    x = Attention_Block(x, skip=1)

    in_channel = out_channel  # 128
    out_channel = in_channel * 4  # 512
    x = Residual_Unit(x, in_channel, out_channel, stride=2)  # 512
    x = Attention_Block(x, skip=1)
    x = Attention_Block(x, skip=1)
    x = Attention_Block(x, skip=1)

    in_channel = out_channel  # 256
    out_channel = in_channel * 4  # 1024
    x = Residual_Unit(x, in_channel, out_channel, stride=1)  # 4x4x1024
    x = Residual_Unit(x, in_channel, out_channel)
    x = Residual_Unit(x, in_channel, out_channel)

    x = AveragePooling2D(pool_size=4, strides=1)(x)  # 1x1x1024
    x = Flatten()(x)

    if dropout:
        x = Dropout(dropout)(x)

    output = Dense(n_classes, kernel_regularizer=l2(regularization), activation='sigmoid')(x)
    model = Model(input_data, output)

    return model

In [ ]:
train_data.shape

(4234, 2)

In [ ]:
model = AttentionResNet92((64,64,1),3,3,3)

In [ ]:
optim = tf.keras.optimizers.Adam (learning_rate=1e-4)
# optim=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 64, 64, 3)            30        ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 64, 64, 3)            12        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 64, 64, 3)            0         ['batch_normalization[0][0

In [ ]:
# Membuat callback untuk menghentikan training apabila sudah mencapai akurasi diatas 90%

class TestCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.90 and logs.get('val_accuracy') >= 0.90):
            print("\nAkurasi telah mencapai >=90%!")
            self.model.stop_training = True

# Definisikan callback untuk menyimpan model terbaik
checkpoint = ModelCheckpoint('model_tubesML_terbaik.h5', save_best_only=True)
callbacks = [TestCallback(), checkpoint]

In [ ]:
init_time = datetime.datetime.now()

train_steps = train_gen.samples // BATCH_SIZE
valid_steps = val_gen.samples // BATCH_SIZE

early_stopping = EarlyStopping(monitor="val_loss", patience=15, mode="min")
checkpoint = ModelCheckpoint("./min_loss.h5", monitor="val_loss", verbose=1,
                             save_best_only=True, save_weights_only=True, mode="min")
learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4,
                                            min_lr=1e-7, verbose=1, mode="min")
csvlogger=tf.keras.callbacks.CSVLogger('./history.csv')

history = model.fit(
    train_gen,
    validation_data=val_gen,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[ callbacks,
                csvlogger,
                checkpoint,
                early_stopping,
                learning_rate_reduction],
    verbose=1,
    class_weight = class_weights
    )

requared_time = datetime.datetime.now() - init_time
print(f'\nRequired time:  {str(requared_time)}\n')

model.save('./Model')

Epoch 1/25
  6/132 [>.............................] - ETA: 7:53 - loss: 11.3932 - accuracy: 0.4323

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt

history=pd.read_csv('./history.csv')

# history_df = history.history
history.loc[0:, ['loss', 'val_loss']].plot()
plt.savefig("loss_val_loss.png")
print("Minimum Validation Loss: {:0.4f}".format(history['val_loss'].min()));

# history_df = pd.DataFrame(history.history)
history.loc[0:, ['accuracy', 'val_accuracy']].plot()
plt.savefig("acc_val_acc.png")
print("Accuracy: {:0.4f}".format(history['val_accuracy'].max()));

In [ ]:
test_steps = test_gen.samples // BATCH_SIZE

In [ ]:
predict = model.predict(test_gen, steps=test_steps)
y_hat = np.argmax(predict, axis=1)
y_hat[:20]

In [ ]:
test_data[['class']]

In [ ]:
test_labels_df = pd.DataFrame()
test_labels_df[['class']] = test_data[['class']]

change = {
'0' : 0,
'1' : 1,
'2' : 2,
}

test_labels_df['class'] = test_labels_df['class'].map(change)
test_labels_df = test_labels_df[ : test_steps*BATCH_SIZE]


y_test = np.array(test_labels_df['class'])
y_test[:20]

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas.util.testing as tm
from sklearn import metrics
import seaborn as sns
sns.set()

plt.rcParams["font.family"] = 'DejaVu Sans'

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greys,
                          save = False):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(b=False)
    if save == True:
      plt.savefig('Confusion Matrix.png', dpi = 300)

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

class_labels=('Normal','Pneumonia','Covid')

cm = metrics.confusion_matrix(y_test, y_hat)



# plot confusin matrix
plt.figure(figsize=(20,10))
plt.grid(False)
plot_confusion_matrix(cm, classes=class_labels, normalize=True, title='Normalized confusion matrix')
plt.savefig("Confusion_Matrix.png")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_hat))

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.keras.utils import to_categorical

class_labels= ['Pneumonia','Normal','COVID-19']

y_pred_original = model.predict(test_gen, steps=test_steps)
y_pred_original.shape

y_test = y_test.astype(int)
y_test.shape

y_pred_original.shape

In [ ]:
from sklearn.metrics import roc_curve, auc

# create plot
fig, c_ax = plt.subplots(1,1, figsize = (10, 10))
for (i, label) in enumerate(class_labels):
    fpr, tpr, thresholds = roc_curve(to_categorical(y_test)[:,i].astype(int), y_pred_original[:,i])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (label, auc(fpr, tpr)))

# Set labels for plot
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
c_ax.set_title('Roc AUC Curve')
plt.savefig("ROC Curve.png")
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

# Untuk menyimpan model
# model.save('model_MLChest.h5')

In [ ]:
# Untuk memuat kembali model
model = load_model('model_MLChest.h5')

OSError: ignored

In [ ]:
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')  # start it
os.environ['DISPLAY']=':1.0'  # tells X clients where to connect to

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

# Function to predict an image given its path
def predict_image(model, image_path):
    labels = ['COVID', 'NORMAL', 'PNEUMONIA']

    img = image.load_img(image_path, target_size=(150, 150))
    img = image.img_to_array(img)
    img = img / 255.0

    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    print(prediction)

    predicted_class = labels[np.argmax(prediction)]
    predicted_probability = np.max(prediction)

    # Display the uploaded image with prediction and probability
    plt.figure(figsize=(4, 4))
    plt.imshow(img[0])  # img[0] to access the image array
    plt.axis('off')  # Sembunyikan sumbu
    plt.title(f'Prediction: {predicted_class}\nProbability: {predicted_probability:.2f}')
    plt.show()

# Function to predict an uploaded image
def predict_uploaded_image(model):
    uploaded = files.upload()
    if uploaded:
        file_path = next(iter(uploaded))
        predict_image(model, file_path)
        print("Image Path:", file_path)

# Load your model (replace 'model_terbaik.h5' with the actual path)
model_path = 'model_MLChest.h5'
model = load_model(model_path)

# Trigger image upload and prediction
predict_uploaded_image(model)


Saving PNEUMONIA_2.png to PNEUMONIA_2.png


ValueError: ignored